# Estimating biomarker ordering

>The sampler for the biomarker ordering can be a bit tricker. The simplest way to do it might be to do a Metropolis-Hastings step where you select two indicies and propose swapping their order. Then you can work out the relative probabilities, evaluate and then accept/reject based on that. It's not the fastest sampler, but it is a lot more straightforward than some ways of doing it.  

In the following, we assume we know the actual $\theta$ and $\phi$ values. Other than those, we know nothing except for participants' observed biomarker values. And we want to estimate the current order in which different biomarkers are affected by the disease in question. 

In [1]:
import pandas as pd 
import numpy as np 
import re 
import altair as alt 
import matplotlib.pyplot as plt 
from collections import Counter

We only have three columns: biomarker, participant, and measurement. 

In [2]:
data = pd.read_csv('data/participant_data.csv')
data.Biomarker = [re.sub("Biomarker ", "", text) for text in data.Biomarker.tolist()]
data_we_have = data.drop(['k_j', 'S_n', 'affected_or_not'], axis = 1)
data_we_have.head()

,Biomarker,participant,measurement
0,0,0,24.714920
1,0,1,32.103408
2,0,2,21.084512
3,0,3,27.019921
4,0,4,27.673772


In [3]:
theta_phi = pd.read_csv('data/means_vars.csv')
theta_phi.head()

,biomarker,theta_mean,theta_var,phi_mean,phi_var
0,0,1.0,0.3,32.0,6.3
1,1,3.0,0.5,31.0,7.4
2,2,5.0,0.2,34.0,9.4
3,3,6.0,1.3,36.0,4.9
4,4,8.0,3.3,38.0,2.5


In [4]:
type(theta_phi['biomarker'][0])

numpy.int64

In [5]:
def fill_up_pdata(pdata, k_j):
    '''Fill up participant data using k_j; basically add two columns: k_j and affected
    Note that this function assumes that pdata already has the S_n column
    Input:
    - pdata: a dataframe of ten biomarker values for a specific participant 
    - k_j: a scalar
    '''
    data = pdata.copy()
    data['k_j'] = k_j
    data['affected'] = data.apply(lambda row: row.k_j >= row.S_n, axis = 1)
    return data 

In [6]:
def compute_single_measurement_likelihood(theta_phi, biomarker, affected, measurement):
    '''Computes the likelihood of the measurement value of a single biomarker
    We know the normal distribution defined by either theta or phi
    and we know the measurement. This will give us the probability
    of this given measurement value. 

    Note that because the likelihood tends to be very very small, 
    we take the natural log of it

    input:
    - theta_phi: the dataframe containing theta and phi values for each biomarker
    - biomarker: an integer between 0 and 9 
    - affected: boolean 
    - measurement: the observed value for a biomarker in a specific participant

    output: a scalar
    '''
    biomarker_params = theta_phi[theta_phi.biomarker == biomarker].reset_index()
    mu = biomarker_params['theta_mean'][0] if affected else biomarker_params['phi_mean'][0]
    var = biomarker_params['theta_var'][0] if affected else biomarker_params['phi_var'][0]
    # sigma = np.sqrt(var)
    likelihood = np.exp(-(measurement - mu)**2/(2*var))/np.sqrt(2*np.pi*var)
    return likelihood

In [7]:
def compute_likelihood(pdata, k_j, theta_phi):
    '''This implementes the formula of https://ebm-book2.vercel.app/distributions.html#known-k-j
    This function computes the likelihood of seeing this sequence of biomarker values for a specific participant
    '''
    data = fill_up_pdata(pdata, k_j)
    likelihood = 1
    for i, row in data.iterrows():
        biomarker = int(row['Biomarker'])
        measurement = row['measurement']
        affected = row['affected']
        likelihood *= compute_single_measurement_likelihood(
            theta_phi, biomarker, affected, measurement)
    return likelihood

## Testing

The above functions can compute the likelihood of a participant's sequence of biomarker data, given that we know the exact ordering and we assume a `k_j`. Next, we will test those functions by selecting a specific participant. We compute the likelihood by trying all possible `k_j` and see whether the one with the highest likelihood is the real `k_j` in the original data. 

In [8]:
p = 15 # we chose this participant
pdata = data[data.participant == p].reset_index(drop=True)
pdata

,Biomarker,participant,measurement,k_j,S_n,affected_or_not
0,0,15,36.547174,6,7,not_affected
1,1,15,2.909218,6,5,affected
2,2,15,5.042268,6,3,affected
3,3,15,37.349589,6,9,not_affected
4,4,15,9.509030,6,1,affected
5,5,15,2.335076,6,2,affected
6,6,15,4.764619,6,6,affected
7,7,15,17.633576,6,10,not_affected
8,8,15,8.029674,6,4,affected
9,9,15,35.276224,6,8,not_affected


In [9]:
# ordering of biomarker affected by the disease
real_ordering_dic = dict(zip(np.arange(10), pdata.S_n))
real_ordering_dic

{0: 7, 1: 5, 2: 3, 3: 9, 4: 1, 5: 2, 6: 6, 7: 10, 8: 4, 9: 8}

In [10]:
# get the participant data without k_j, S_n, and affected or not
pdata = data_we_have[data_we_have.participant == p].reset_index(drop=True)
# obtain real ordering:
pdata['S_n'] = pdata.apply(lambda row: real_ordering_dic[int(row['Biomarker'])], axis = 1)
pdata

,Biomarker,participant,measurement,S_n
0,0,15,36.547174,7
1,1,15,2.909218,5
2,2,15,5.042268,3
3,3,15,37.349589,9
4,4,15,9.509030,1
5,5,15,2.335076,2
6,6,15,4.764619,6
7,7,15,17.633576,10
8,8,15,8.029674,4
9,9,15,35.276224,8


In [11]:
num_biomarkers = len(pdata.Biomarker.unique())
# calculate likelihood for all possible k_j
likelihood_list = [
    compute_likelihood(pdata=pdata, k_j=x, theta_phi=theta_phi) for x in range(num_biomarkers+1)]
kjs = np.arange(11)
dic = dict(zip(kjs, likelihood_list))
df = pd.DataFrame.from_dict(dic, orient='index', columns=['likelihood']).reset_index()
df.sort_values('likelihood', ascending=False)

,index,likelihood
6,6,8.498626e-15
5,5,1.070326e-36
4,4,1.961882e-60
3,3,2.230992e-80
2,2,1.390695e-100
1,1,9.775606e-150
0,0,4.941208e-220
7,7,0.000000e+00
8,8,0.000000e+00
9,9,0.000000e+00


<!-- From the above result, we can see that the most likelihood `k_j` is 8, which is in fact the real `k_j` in the participant data.  -->

## Metropolis-Hastings Algorithm Implementation

Next, we will implement the metropolis-hastings algorithm using the above functions. 

In [12]:
def average_all_likelihood(pdata, num_biomarkers, theta_phi):
    '''This is to compute https://ebm-book2.vercel.app/distributions.html#unknown-k-j
    '''
    return np.mean([compute_likelihood(pdata=pdata, k_j=x, theta_phi=theta_phi) for x in range(num_biomarkers+1)])

In [13]:
def compute_likelihood_based_on_ordering(ordering, data, num_participants, num_biomarkers, theta_phi):
    """Compute the likelihood of all participants having their data
    Inputs:
        - ordering: an array of ordering for biomarker 0-9
        - data: data_we_have
        - num_participants
        - num_biomarkers 
    Outputs:
        - likelihood
    """
    # biomarker - order dict
    ordering_dic = dict(zip(np.arange(num_biomarkers), ordering))
    # fill up S_n column using the ordering dict
    # copy first in order not to change data_we_have
    filled_data = data.copy()
    filled_data['S_n'] = filled_data.apply(lambda row: ordering_dic[int(row['Biomarker'])], axis = 1)
    likelihood = 0 
    for p in range(num_participants):
        pdata = filled_data[filled_data.participant == p].reset_index(drop=True)
        average_likelihood = average_all_likelihood(pdata, num_biomarkers, theta_phi)
        # print(average_likelihood)
        if average_likelihood == 0:
            # this is to avoid np.log(0)
            log_likelihood = np.log(average_likelihood + 1e-20)
        else:
            log_likelihood = np.log(average_likelihood)
            # print(log_likelihood)
        likelihood += log_likelihood
    return likelihood

In [14]:
def metropolis_hastings(data, iterations, theta_phi):
    '''Implement the metropolis-hastings algorithm
    Inputs: 
        - data: data_we_have
        - iterations: number of iterations

    Outputs:
        - best_order: a numpy array
        - best_likelihood: a scalar 
    '''
    num_participants = len(data.participant.unique())
    num_biomarkers = len(data.Biomarker.unique())

    # initialize an ordering and likelihood
    best_order = np.arange(num_biomarkers)
    best_likelihood = -np.inf 
    # best_order = np.array(list(real_ordering_dic.values()))
    # best_likelihood = compute_likelihood_based_on_ordering(
    #     best_order, data, num_participants, num_biomarkers, theta_phi
    # )
    for _ in range(iterations):
        new_order = best_order.copy()
        # randomly select two indices
        a, b = np.random.choice(num_biomarkers, 2, replace=False)
        # swaping the order
        new_order[a], new_order[b] = new_order[b], new_order[a]
        likelihood = compute_likelihood_based_on_ordering(new_order, data, num_participants, num_biomarkers, theta_phi)
        if likelihood > best_likelihood:
            best_likelihood = likelihood 
            best_order = new_order
        else: 
            ratio = likelihood/best_likelihood
            random_number = np.random.rand()
            if random_number < ratio:
                best_likelihood = likelihood
                best_order = new_order
        print(f"iteration {_ + 1} done")
    return best_order, best_likelihood


In [15]:
best_order, best_likelihood = metropolis_hastings(data_we_have, 10, theta_phi)

iteration 1 done


iteration 2 done


iteration 3 done


iteration 4 done


iteration 5 done


iteration 6 done


iteration 7 done


iteration 8 done


iteration 9 done


iteration 10 done


In [16]:
best_order, np.array(list(real_ordering_dic.values()))

(array([0, 9, 4, 6, 3, 2, 1, 7, 8, 5]),
 array([ 7,  5,  3,  9,  1,  2,  6, 10,  4,  8]))

## Unknown theta and phi

I found it challenging to infer the ordering of biomarkers affected by the disease without knowing theta and phi. This is because we do not need to now participants' real disease stage in the formula of https://ebm-book2.vercel.app/distributions.html#unknown-k-j, however, without knowing all participants' disease stages, we are not able to estimate theta and phi. 

In [17]:
data.head()

,Biomarker,participant,measurement,k_j,S_n,affected_or_not
0,0,0,24.714920,6,7,not_affected
1,0,1,32.103408,0,7,not_affected
2,0,2,21.084512,4,7,not_affected
3,0,3,27.019921,4,7,not_affected
4,0,4,27.673772,1,7,not_affected


In [18]:
actual_stage_dict = dict(zip(data.participant, data.k_j))
actual_stages = np.array(list(actual_stage_dict.values()))
actual_stages

array([ 6,  0,  4,  4,  1,  1,  4,  0,  1,  5,  6,  0,  7,  1,  2,  6,  5,
        9,  9,  4,  8,  9, 10,  5,  7,  9, 10,  8,  7,  5,  4,  7,  3,  4,
        0,  9,  9,  8, 10,  6,  0,  0,  3, 10,  4, 10,  1,  6,  9,  4,  2,
        6,  3, 10,  3,  0,  1,  2,  1,  5,  8,  4,  2,  9,  1,  7,  7,  0,
        7,  8,  9,  8,  7,  2,  2,  3, 10,  9, 10,  1,  4,  7,  0,  4, 10,
        4,  8,  1,  2,  7,  2,  3,  1,  3,  8,  7,  6,  1,  2,  6])

In [19]:
data_we_have.head()

,Biomarker,participant,measurement
0,0,0,24.714920
1,0,1,32.103408
2,0,2,21.084512
3,0,3,27.019921
4,0,4,27.673772


In [20]:
def add_kj_and_affected(data_we_have, participant_stages, num_participants):
    '''This is to fill up data_we_have. 
    Basically, add two columns: k_j, and affected, based on the initial or updated participant_stages
    
    Inputs:
        - data_we_have
        - participant_stages: np array 
        - participants: 0-199
    '''
    participant_stage_dic = dict(zip(np.arange(0,num_participants), participant_stages))
    data_we_have['k_j'] = data_we_have.apply(lambda row: participant_stage_dic[row.participant], axis = 1)
    data_we_have['affected'] = data_we_have.apply(lambda row: row.k_j >= row.S_n, axis = 1)
    return data_we_have 

In [21]:
def estimate_params_exact(m0, n0, s0_sq, v0, data):
    '''This is to estimate means and vars based on conjugate priors
    Inputs:
        - data: a vector of measurements for a specific biomarker in a specific group (affected or not)
        - m0: prior estimate of $\mu$.
        - n0: how strongly is the prior belief in $m_0$ is held.
        - s0_sq: prior estimate of $\sigma^2$.
        - v0: prior degress of freedome, influencing the certainty of $s_0^2$.
    
    Outputs:
        - mu estiate, var estimate
    '''
    # Data summary
    sample_mean = np.mean(data)
    sample_size = len(data)
    sample_var = np.var(data, ddof=1)  # ddof=1 for unbiased estimator

    # Update hyperparameters for the Normal-Inverse Gamma posterior
    updated_m0 = (n0 * m0 + sample_size * sample_mean) / (n0 + sample_size)
    updated_n0 = n0 + sample_size
    updated_v0 = v0 + sample_size 
    updated_s0_sq = (1 / updated_v0) * ((sample_size - 1) * sample_var + v0 * s0_sq + 
                    (n0 * sample_size / updated_n0) * (sample_mean - m0)**2)
    updated_alpha = updated_v0/2
    updated_beta = updated_v0*updated_s0_sq/2

    # Posterior estimates
    mu_posterior_mean = updated_m0
    sigma_squared_posterior_mean = updated_beta/updated_alpha

    return mu_posterior_mean, sigma_squared_posterior_mean

In [22]:
def get_estimated_theta_phi(num_biomarkers, data_we_have):
    '''To get estimated parameters, returns a Pandas DataFrame
    Input:
    - num_biomarkers: 10
    - data_we_have: full data filled up by S_n, k_j and affected

    Output: 
    - estimate_means_vars_df, just like means_vars_df, containing the estimated mean and var for 
      distribution of biomarker values when the biomarker is affected and not affected
    '''
    # empty list of dictionaries to store the estimations
    means_vars_estimate_dict_list = []
    # 0 - 9
    for biomarker in range(num_biomarkers): 
        dic = {'biomarker': biomarker}  # Initialize dictionary outside the inner loop
        for affected in [True, False]:
            data_full = data_we_have[(data_we_have.Biomarker == str(biomarker)) & (
            data_we_have.affected == affected)]
            data = np.array(data_full.measurement)
            # if len(data) == 0:
            #     print(data_full)
            mu_estimate, var_estimate = estimate_params_exact(
                m0 = 0, n0 = 1, s0_sq = 1, v0 = 1, data=data)
            if affected:
                dic['theta_mean'] = mu_estimate
                dic['theta_var'] = var_estimate
            else:
                dic['phi_mean'] = mu_estimate
                dic['phi_var'] = var_estimate
        means_vars_estimate_dict_list.append(dic)
    estimate_theta_phi = pd.DataFrame(means_vars_estimate_dict_list)
    return estimate_theta_phi

In [23]:
def metropolis_hastings_unknown_theta_phi(data_we_have, iterations):
    num_participants = len(data_we_have.participant.unique())
    num_biomarkers = len(data_we_have.Biomarker.unique())

    # initialize an ordering and likelihood
    best_order = np.arange(num_biomarkers)
    best_likelihood = -np.inf 

    for _ in range(iterations):
        new_order = best_order.copy()
        # randomly select two indices
        a, b = np.random.choice(num_biomarkers, 2, replace=False)
        # swaping the order
        new_order[a], new_order[b] = new_order[b], new_order[a]

        # likelihood of seeing all participants' data 
        # biomarker - order dict
        ordering_dic = dict(zip(np.arange(num_biomarkers), new_order))
        # fill up S_n column using the ordering dict
        # copy first in order not to change data_we_have
        data = data_we_have.copy()
        # now data_we_have has S_n column
        data['S_n'] = data.apply(lambda row: ordering_dic[int(row['Biomarker'])], axis = 1)

        # initialize participant_stages 
        # note that high should be num_stages + 1; otherwise, no participants will be in the stage of 10
        participant_stages = np.random.randint(low = 0, high = num_biomarkers+1, size = num_participants)
        if np.any(participant_stages == 0):
            print("0 is in stage_likelihood")
        else:
            print("0 is not in stage_likelihood")

        # add kj and affected
        data = add_kj_and_affected(data, participant_stages, num_participants)

        # get estimated_theta_phi
        estimated_theta_phi = get_estimated_theta_phi(num_biomarkers, data_we_have=data)

        all_participant_likelihood = 0 
        for p in range(num_participants):
            pdata = data[data.participant == p].reset_index(drop=True)
            # initiaze stage_likelihood
            stage_likelihood = np.zeros(num_biomarkers + 1)
            for k_j in range(num_biomarkers +1):
                # even though data above has everything, it is filled up by random stages
                # we don't like it and want to know the true k_j
                pdata_with_this_kj = fill_up_pdata(pdata=pdata, k_j=k_j)
                # likelihood for this participant to have this sequence of biomarker values
                participant_likelihood = 1
                for i, row in pdata_with_this_kj.iterrows():
                    biomarker = int(row['Biomarker'])
                    measurement = row['measurement']
                    affected = row['affected']
                    participant_likelihood *= compute_single_measurement_likelihood(
                        estimated_theta_phi, biomarker, affected, measurement)
                stage_likelihood[k_j] = participant_likelihood
            likelihood_sum = np.sum(stage_likelihood)
            normalized_stage_likelihood = [l/likelihood_sum for l in stage_likelihood]
            sampled_stage = np.random.choice(np.arange(num_biomarkers + 1), p = normalized_stage_likelihood)
            participant_stages[p] = sampled_stage     

            # formula here: https://ebm-book2.vercel.app/distributions.html#unknown-k-j
            average_likelihood = np.mean(stage_likelihood)
            if average_likelihood == 0:
                print(f'averge likelihood is zero')
                log_average_likelihood = np.log(average_likelihood + 1e-20)
            else:
                log_average_likelihood = np.log(average_likelihood)
            all_participant_likelihood += log_average_likelihood
        if np.any(participant_stages == 0):
            print("0 is in participant stages")
        else:
            print("0 is not in participant stages")
        if  all_participant_likelihood > best_likelihood:
            best_likelihood = all_participant_likelihood 
            best_order = new_order
        else: 
            ratio = all_participant_likelihood/best_likelihood
            random_number = np.random.rand()
            if random_number < ratio:
                best_likelihood = all_participant_likelihood
                best_order = new_order
        print(f"iteration {_ + 1} done")
    return best_order, best_likelihood


In [24]:
best_order, best_likelihood = metropolis_hastings_unknown_theta_phi(data_we_have, iterations = 2)

0 is in stage_likelihood


/Users/hongtaoh/anaconda3/envs/bayes/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/hongtaoh/anaconda3/envs/bayes/lib/python3.8/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/Users/hongtaoh/anaconda3/envs/bayes/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3715: RuntimeWarning: Degrees of freedom <= 0 for slice
  return _methods._var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/Users/hongtaoh/anaconda3/envs/bayes/lib/python3.8/site-packages/numpy/core/_methods.py:223: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean, casting='unsafe',
/Users/hongtaoh/anaconda3/envs/bayes/lib/python3.8/site-packages/numpy/core/_methods.py:257: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


0 is in participant stages
iteration 1 done
0 is in stage_likelihood


/Users/hongtaoh/anaconda3/envs/bayes/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/hongtaoh/anaconda3/envs/bayes/lib/python3.8/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/Users/hongtaoh/anaconda3/envs/bayes/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3715: RuntimeWarning: Degrees of freedom <= 0 for slice
  return _methods._var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/Users/hongtaoh/anaconda3/envs/bayes/lib/python3.8/site-packages/numpy/core/_methods.py:223: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean, casting='unsafe',
/Users/hongtaoh/anaconda3/envs/bayes/lib/python3.8/site-packages/numpy/core/_methods.py:257: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


0 is in participant stages
iteration 2 done


In [25]:
best_order

array([0, 1, 5, 3, 4, 2, 7, 6, 8, 9])